In [1]:
import pymysql
import csv
import boto3
import configparser

In [5]:
parser = configparser.ConfigParser()
parser.read("pipeline.conf.txt")
hostname = parser.get("mysql_config", "hostname")
port = parser.get("mysql_config","port")
username = parser.get("mysql_config", "username")
password = parser.get("mysql_config","password")
dbname = parser.get("mysql_config","database")

In [6]:
conn = pymysql.connect(host = hostname,
                      port = int(port),
                      user = username,
                      password = password,
                      db=dbname)

if conn is None:
    print("Error connecting to SQL db")
else:
    print("Connection Established")

Connection Established


## Full Table Load

In [7]:
m_query = "select * from box;"
local_filename = "box_extract.csv"

m_cursor = conn.cursor()
m_cursor.execute(m_query)
results = m_cursor.fetchall()

with open(local_filename, "w") as fp:
    csv_w = csv.writer(fp, delimiter='|')
    csv_w.writerows(results)

fp.close()
m_cursor.close()
conn.close()

## Write to S3

In [11]:
parser = configparser.ConfigParser()
parser.read("pipeline.conf.txt")
access_key = parser.get("aws_boto_credentials","access_key")
secret_key = parser.get("aws_boto_credentials", "secret_key")
bucket_name = parser.get("aws_boto_credentials", "bucket")

s3 = boto3.client('s3', aws_access_key_id = access_key, aws_secret_access_key = secret_key)

s3_file = local_filename

s3.upload_file(local_filename, bucket_name, s3_file)